In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [64]:
#import manipulation packages
import pandas as pd
import numpy as np

#import visualization packages
import matplotlib.pyplot as plt
import seaborn as sns

#to see all the columns without Jupyter redacting them 
pd.set_option('display.max_columns', None)

#import packages for data modeling
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,\
confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from xgboost import plot_importance

import pickle


In [65]:
#import datasets
train = '/kaggle/input/titanic/train.csv'
test = '/kaggle/input/titanic/test.csv'
submission = '/kaggle/input/titanic/gender_submission.csv'

dfo = pd.read_csv(train)
df1 = pd.read_csv(test)
df2 = pd.read_csv(submission)

dfo.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [66]:
dfo.shape

(891, 12)

In [67]:
dfo.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [68]:
dfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [69]:
dfo.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [70]:
df_data = dfo.copy()

In [71]:
df_data = df_data.drop("Name", axis=1)

In [72]:
df_data = df_data.drop("Sex", axis=1)

In [73]:
df_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,35.0,0,0,373450,8.0500,NaN,S


In [74]:
df_data_dummies = pd.get_dummies(df_data, columns = ['Survived', 'PassengerId'])

In [75]:
import pandas as pd
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [76]:
print(test.columns)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [77]:
train = train.dropna()
test = test.ffill()

In [78]:
#Define features and target
X = train.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train["Survived"]
X = test.drop(["PassengerId"], axis=1)

# Split into train/validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train.drop(["Survived", "Sex", "Embarked", "Name", "Ticket", "Cabin"], axis=1),
    train["Survived"],
    test_size=0.2,
    random_state=42
)

# Drop from test set too
X_test = test.drop(["Sex", "Embarked", "PassengerId","Name", "Ticket", "Cabin"], axis=1)

In [79]:
print(X.columns)

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')


In [80]:
#X = X.drop(columns=['Embarked', 'Name', 'Cabin'], errors='ignore')
#X_test = X_test.drop(columns=['Embarked','Name', 'Cabin','Ticket'], errors='ignore')

# Define features (drop same columns for both train and test)
drop_cols = ["Survived", "Sex", "Embarked", "Name", "Ticket", "Cabin"]

X_train = train.drop(drop_cols, axis=1)
y_train = train["Survived"]

X_test = test.drop(["Sex", "Embarked", "Name", "Ticket", "Cabin"], axis=1)

In [81]:
# Align train and test so they have the same columns
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)

# Do the same preprocessing for X_test
X_test['Age'] = X_test['Age'].fillna(X['Age'].median())
X_test['Fare'] = X_test['Fare'].fillna(X['Fare'].median())

# Double-check test set
print(X_test.dtypes)

Pclass        int64
Name          int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket        int64
Fare        float64
Cabin         int64
Embarked      int64
dtype: object


In [82]:
xgb_model = XGBClassifier(enable_categorical=True, random_state=42)

In [83]:
print("Shape of X after cleaning:", X.shape)

Shape of X after cleaning: (418, 10)


In [84]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print(X.head())

Shape of X: (418, 10)
Shape of y: (183,)
   Pclass                                          Name     Sex   Age  SibSp  \
0       3                              Kelly, Mr. James    male  34.5      0   
1       3              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1   
2       2                     Myles, Mr. Thomas Francis    male  62.0      0   
3       3                              Wirz, Mr. Albert    male  27.0      0   
4       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1   

   Parch   Ticket     Fare Cabin Embarked  
0      0   330911   7.8292   NaN        Q  
1      0   363272   7.0000   NaN        S  
2      0   240276   9.6875   NaN        Q  
3      0   315154   8.6625   NaN        S  
4      1  3101298  12.2875   NaN        S  


In [85]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [86]:
# Split train/validation sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

In [87]:
xgb = XGBClassifier(objective='binary:logistic', random_state=0)

In [88]:
#model.fit(X_train, y_train)

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [89]:
# Evaluate
accuracy = model.score(X_val, y_val)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 1.0


In [90]:
cv_params = {'max_depth': [4, 6],
 'min_child_weight': [3, 5],
 'learning_rate': [0.1, 0.2, 0.3],
 'n_estimators': [5,10,15],
 'subsample': [0.7],
 'colsample_bytree': [0.7]
 }

In [91]:
%%time
# fit the GridSearch model to training data
xgb = xgb.fit(X_train, y_train)
xgb

CPU times: user 178 ms, sys: 5.38 ms, total: 183 ms
Wall time: 48.8 ms


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [92]:
y_test_pred = model.predict(X_test)


In [93]:
print("✅ Predictions shape:", y_test_pred.shape)

✅ Predictions shape: (418,)


In [94]:
# Create submission
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': y_test_pred
})

In [95]:
submission.to_csv("submission.csv", index=False)
print("my submission.csv is ready!")

my submission.csv is ready!
